## 05 - 02 Exploratory Data Analysis using Pandas
Based on the concepts that we have built in the last couple of modules, we will explore the imdb dataset but before we do that, let's first look at ways of loading the datasets as pandas dataframe.

### Loading Data
You can read data from a CSV file using the read_csv function. By default, it assumes that the fields are comma-separated.
Pandas supports following file formats:

|Function| Description|
|:---|:---|
|read_csv | Load delimited data from a file, URL, or file-like object. Use comma as default delimiter|
|read_table | Load delimited data from a file, URL, or file-like object. Use tab ('\t') as default delimiter|
|read_fwf | Read data in fixed-width column format (that is, no delimiters)|
|read_clipboard | Version of read_table that reads data from the clipboard. Useful for converting tables from web pages.|

For most of the analysis work, you will be loading the csv, tsv or some delimited files. We will only look at `read_csv` function in this example.

First, let's import the dataset:


In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-darkgrid')

Lets load the cast, release-dates and titles dataset. This might take sometime.

In [5]:
%%time
cast = pd.DataFrame.from_csv('./sample_datasets/cast.csv', index_col=None)
release_dates = pd.read_csv('./sample_datasets/release_dates.csv', index_col=None,
                            parse_dates=['date'], infer_datetime_format=True)
titles = pd.DataFrame.from_csv('./sample_datasets/titles.csv', index_col=None)

/usr/local/Cellar/ipython/7.2.0/libexec/vendor/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  """Entry point for launching an IPython kernel.


FileNotFoundError: File b'./sample_datasets/cast.csv' does not exist

Lets look at some of the contents of these dataframes

In [6]:
cast.head()

NameError: name 'cast' is not defined

> The `n` in the cast dataframe tells us the rank or the order of the leading roles.

In [ ]:
release_dates.head()

In [ ]:
titles.head()

Do you know how many movies released since you were born?

In [ ]:
my_birth_year = 1990
len(titles[(titles['year']>my_birth_year) & (titles['year']<2017)])

Lets plot how many movies released every year since you were born

In [ ]:
titles.groupby('year').size().loc[my_birth_year:2016].plot(title="Number of movies released every year")

Pandas provides matplotlib based plotting as a part of dataframe. To plot any dataframe (with *plottable data*) you can simply use `.plot()`.

Ofcourse we can also do this manually using matplotlib's pyplot as well (which is actually what pandas uses under the cover)

In [ ]:
fig, ax = plt.subplots()
ax.plot(titles.groupby('year').size().loc[my_birth_year:2016])
ax.set_xlabel("Year")
ax.set_ylabel("# of Movies")
ax.set_title("Number of movies released every year")
ax.set_xlim(1990, 2016)

Hmm.. How about the total number of movies released every decade?

This will involve a little math. We know that the `titles` dataframe has a `year` column which, obviously, has the year when a movie was released.

To count the occurence of something, we can use the `value_counts` method. 

So now, all we need to do is pass the *decade* as key to the `titles` dataframe and apply the `value_counts` method. 

Let's see how to do this

In [ ]:
(titles['year'] // 10 * 10).value_counts().sort_index().plot(kind='bar')

How many movies did a *movie star* star in?

In [ ]:
movie_star = "Matt Damon"
len(cast[cast['name'] == movie_star])

What are the 10 most common name of the roles played by characters?

In [ ]:
cast['character'].value_counts().head(10)

What are the 10 most common movie names?

In [ ]:
titles['title'].value_counts().head(10)

Similarly, you can find who has been the most in the movies

In [ ]:
cast['name'].value_counts().head(10)

Lets find the years when *The Bourne* series were released.

In [ ]:
titles[titles['title'].str.contains("Bourne")].sort_values('year')

Lets find the movies when *Matt Damon* was *Jason Bourne*

In [ ]:
matt_jbourne = cast[(cast['title'].str.contains("Bourne")) & 
                    (cast['name'].str.contains("Matt Damon"))]
matt_jbourne

So, How many movies do you think is released every year starring Matt Damon?

In [ ]:
fig, ax = plt.subplots()
ax.plot(cast[cast['name'] == "Matt Damon"].groupby('year').size())
ax.set_xlabel("Year")
ax.set_ylabel("# of Movies")
ax.set_title("Matt Damon movies")

And how about the ranks at which *Matt Damon* stars in the movies?

In [ ]:
matt_movies = cast[cast.name == 'Matt Damon'].sort_values('year')
matt_movies = matt_movies[matt_movies['n'].notnull()]
# For scatter plots, you can simply pass the column
# names for the x and y argument
matt_movies.plot(x='year', y='n', kind='scatter')

So how many leading roles?

In [ ]:
matt_movies = cast[cast.name == 'Matt Damon'].sort_values('year')
matt_movies[matt_movies['n'] == 1]['n'].value_counts()

Lets see how many people were casted in all these *Jason Bourne* movies

In [ ]:
%%time
cast[(cast['title'].str.contains("Bourne"))].groupby(['year', 'title']).size()

Among these casts, how many were actors and actresses?

In [ ]:
%%time 
cast[(cast['title'].str.contains("Bourne"))].groupby(['year', 'title', 'type']).size()

Lets plot the above and see the total number of roles based on gender

In [ ]:
jason_cast = cast[(cast['title'].str.contains("Bourne"))]
jason_cast_gender = jason_cast[['year', 'type']].groupby(['year', 'type']).size().unstack()
print(jason_cast_gender)
jason_cast_gender.plot()

Lets find out the entire cast of the *The Bourne Ultimatum* and print just the top 10 leads

In [ ]:
cast[cast['title'] == "The Bourne Ultimatum"].sort_values(['n']).head(10)

Lets see in what months Matt Damon's movies are most often released in the USA.

- First, find the year and the title of unique movies starring Matt Damon were released

In [ ]:
matt_movies = cast[cast['name'] == "Matt Damon"][['title', 'year']].drop_duplicates()

Now we have to re-index the `release_dates` dataframe with `title` and `year` keys

In [ ]:
rel_dts = release_dates.set_index(['title', 'year']).sort_index()

- Now, the 'month' part is present in `date` column present in `release_dates` dataframe so we have to combine the `cast` and `release_dates` dataframes for Matt Damon's movies by some common index (in our case, we will use the common index as `title` and `year` since it is present in both the dataframes)

In [ ]:
matt_movie_releases = matt_movies.join(rel_dts, on=['title', 'year'])

- We only want movies released in the USA

In [ ]:
matt_movie_releases = matt_movie_releases[matt_movie_releases['country'] == "USA"]

- Now lets plot the `month` part.

In [ ]:
matt_movie_releases['date'].dt.month.value_counts().sort_index().plot(kind='bar')

Cool, Now lets see when the Bourne movie series were released in different countries.
> Since there are is a huge list of countries, lets just select "USA", "UK" and "India"

In [ ]:
countries = ["USA", "UK", "Australia"]
matt_movie_releases = matt_movies.join(rel_dts, on=['title', 'year'])
matt_movie_countries = matt_movie_releases[matt_movie_releases['country'].str.contains('|'.join(countries))]
matt_movie_countries.set_index(['title', 'country'])[['date']].unstack()

> To match the `country` column against all the elements of the list, we use the `|` (OR) operator. It is considered as a Regular Expression.

We can also create a `Pivot` table to provide the above output.
> - The pivot table takes simple column-wise data as input, and groups the entries into a two-dimensional table that provides a multidimensional summarization of the data.
> - Think of it as a multi-dimensional GroupBy function

In [ ]:
countries = ["USA", "UK", "Australia"]
matt_movie_releases = matt_movies.join(rel_dts, on=['title', 'year'])
matt_movie_countries = matt_movie_releases[matt_movie_releases['country'].str.contains('|'.join(countries))]
matt_movie_countries.pivot(index='title', columns='country', values='date')

Do you know when are the most *Action* movies released in the USA?

In [ ]:
action_usa = release_dates[(release_dates['title'].str.contains('Action')) & 
                           (release_dates['country'] == "USA")]
action_usa['date'].dt.dayofweek.value_counts().sort_index().plot(kind='bar')

In which months are *Matt Damon*'s movies generally released in the USA?

In [ ]:
matt_movies = cast[cast['name'] == 'Matt Damon']
matt_movies_usa = matt_movies.merge(release_dates[release_dates.country == 'USA']).sort_values('date')
matt_movies_usa.date.dt.month.value_counts().sort_index().plot(kind='bar')

The above examples should give you some idea about the importance of Pandas and how its high level functions mask the complex computation that is performed on the underlying Numpy arrays.

This is by no means an exhaustive list of all the functions. We have barely scratched the surface. 

> Remember --
> ### The only way to become a master of something is to be really With It! 
So keep practicing and whenever you are stuck:
- Go through the official documentation.
- Enter the object name and Press `<TAB>` or `.?` and Jupyter will show you the docstring.
- Don't trust the examples blindly. Run them, modify them, make mistakes and then rectify them.
- Don't simply copy the StackOverflow or StackExchange or answers from anywhere else. Understand the solution that you find on such sites and then and only then use it in your code.

>If you want more tutorials/ cookbooks, take a look at 

> - [`Pandas own 10 minute to Pandas`](http://pandas.pydata.org/pandas-docs/stable/10min.html#min '10 minutes to pandas')

> - [`Hernan Rojas's Learn Pandas`](https://bitbucket.org/hrojas/learn-pandas 'hrojas's Learn Pandas')

> - [`Pandas Cookbook`](http://pandas.pydata.org/pandas-docs/stable/cookbook.html#cookbook 'Pandas Cookbook')

> - [`Greg Reda's Blog`](http://www.gregreda.com/2013/10/26/intro-to-pandas-data-structures/ 'Greg Redas blog on Pandas')
